In [2]:
import numpy as np
import pandas as pd
from numpy import ma
from numpy import linalg as LA

In [50]:
class AgglomerativeClustering():
    def __init__(self, metric='euclidean', linkage='single'):
        metric_choices = {
            'euclidean': lambda a, b: np.sqrt(((a-b) ** 2).sum()),
            'precomputed': None
        }
        try:
            metric_func = metric_choices[metric]
        except KeyError as e:
            raise ValueError(
                f"Unknown metric."
            ) from e
        self._metric_func = metric_func

        def compute_ward_dist(a, b):
            c = np.concatenate(( a, b ))
            ward_dist = metric_func(c, c.mean(0)) ** 2 \
                - metric_func(b, b.mean(0)) ** 2 \
                - metric_func(a, a.mean(0)) ** 2
            return ward_dist
        linkage_choices = {
            'single': lambda a, b: np.minimum(a, b),
            'average': lambda a, b: (a+b)/2,
            'complete': lambda a, b: np.maximum(a, b),
            'ward': compute_ward_dist
        }
        try:
            join_func = linkage_choices[linkage]
        except KeyError as e:
            raise ValueError(
                f"Unknown linkage. Should be one of {linkage_choices.keys()}"
            ) from e
        if linkage == 'ward' and metric != 'euclidean':
            raise ValueError(
                f"Only euclidean disance is availible for ward's method."
            )
        self._join_func = join_func

        self.distance_matrix = None

    def __compute_distance_matrix(self, X):
        N = X.shape[0]
        matrix = np.empty((N, N))
        for i in range(N):
            for j in range(N):
                dist = self._metric_func(X[i], X[j])
                matrix[i,j] = dist
        self.distance_matrix = matrix

    def fit(self, X):
        if self._metric_func != None:
            self.__compute_distance_matrix(X)
        else:
            self.distance_matrix = X
        
        agglomerative_schedule = []

        D = self.distance_matrix.copy()
        N = D.shape[0]
        vertex_index = N
        cluster_mapping = np.arange(N, dtype=int)

        mask = np.zeros_like(D)
        D_masked = ma.masked_array(D, mask)
        D_masked.fill_value = np.inf
        np.fill_diagonal(D_masked.mask, 1)

        for k in range(N-1):
            ind_min_flat = np.argmin(D_masked)
            ind_min = np.unravel_index(ind_min_flat, D_masked.shape)
            i, j = ind_min
            a = D[:, i].copy()
            b = D[:, j].copy()
            new_cluster = self._join_func(a, b)
            
            agglomerative_schedule.append((i+1, j+1,
                D_masked[ind_min].copy()))

            D_masked[:, [j]] = ma.masked
            D_masked[[j], :] = ma.masked
            D_masked.data[:, [i]] = np.atleast_2d(new_cluster).T
            D_masked.data[[i], :] = new_cluster
        self.agglomerative_schedule =  agglomerative_schedule

In [51]:
np.set_printoptions(precision=2)
pd.set_option('display.precision', 2)

X1 = pd.read_excel('data/clustering.xlsx', sheet_name='raw_data')
X1

,x1,x2
0,1,5
1,7,9
2,1,3
3,9,7


In [52]:
model = AgglomerativeClustering(linkage='ward')
model.fit(X1.to_numpy())
print(*model.agglomerative_schedule, sep='\n')

(1, 3, 2.0)
(1, 2, 0.4862160718316275)
(1, 4, 12.14027664890245)


In [48]:
X2 = pd.read_excel('data/clustering.xlsx', sheet_name='dist_matrix', header=None)
X2

,0,1,2,3,4
0,0.0,2.2,3.0,5.1,5.8
1,2.2,0.0,1.4,5.0,6.4
2,3.0,1.4,0.0,6.4,7.8
3,5.1,5.0,6.4,0.0,2.0
4,5.8,6.4,7.8,2.0,0.0


In [43]:
model = AgglomerativeClustering(metric='precomputed')
model.fit(X2.to_numpy())
print(*model.agglomerative_schedule, sep='\n')

(2, 3, 1.4)
(4, 5, 2.0)
(1, 2, 2.2)
(1, 4, 5.0)
